# Convert IHS materials data to consistent units and add uncertainties

In [ ]:
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None

In [ ]:
# Data file paths
input_path = '../data/'
output_path = '../data/processed/'
ihs_materials_path = input_path+'extracted/ihsMaterials.csv'
unit_conversion_path = input_path+'extra_inputs/unit_conversions.xlsx'

In [ ]:
# Read in materials
use_columns = ['Code', 'Data Version', 'Source', 'Source type', 'Target', 'Research Year', 'Geography', 'Product', 'Value', 'Value unit', 'Capacity unit']
ihs_region = 'Germany'

materials = pd.read_csv(ihs_materials_path)
materials = materials[use_columns].loc[materials['Geography'] == ihs_region]
materials.rename(columns={'Source':'Source/Object', 'Source type':'Type', 'Target':'Target/Process', 'Geography':'Country/Reg'}, inplace=True)
materials['MeasType'] = materials['Type'].replace({'Raw Material':'Recipe', 'By-Product':'Recipe'})
materials['Provenance'] = 'IHS PEP'

# Enforce negative values as by-products
materials['Type'] = ['By-Product' if val < 0 else def_type for val, def_type in zip(materials['Value'], materials['Type'])]

materials.head()

In [ ]:
materials['Product'].unique()

In [ ]:
# Define unit conversion function

def convert_units(mat_df:pd.DataFrame, unit_conversion:pd.DataFrame, unit_name:str, unit_column:str, value_column:str, unit_name_in_col:str, target_unit:str, inv_density:bool=False, standard_conversion:int=1) -> pd.DataFrame:

    # Get appropriate conversions for unit
    unit_filts = unit_conversion[unit_conversion['Unit']==unit_name]
    unit_dict = dict(zip(unit_filts['Name'], unit_filts['density']))

    # Cycle through dataframe to get conversion factor for each row
    unit_mats = mat_df[mat_df[unit_column] == unit_name_in_col]
    conv_factor = np.zeros(len(unit_mats))
    for i, target in enumerate(unit_mats[value_column]):
        for type in unit_dict.keys():
            if target.startswith(type):
                conv_factor[i] = unit_dict[type]
                break
            if type == list(unit_dict.keys())[-1]:
                raise Exception(unit_name+' for '+target+' unit conversions missing')

    # Convert values
    if inv_density: # If converting from value in units to weight
        unit_mats['Value'] = (unit_mats['Value']/standard_conversion)/conv_factor
        #unit_mats['Value_sigma'] = (unit_mats['Value_sigma']/standard_conversion)/conv_factor
    else: # If converting proportion of other materials
        unit_mats['Value'] = (unit_mats['Value']/standard_conversion)*conv_factor
        #unit_mats['Value_sigma'] = (unit_mats['Value_sigma']/standard_conversion)*conv_factor
    unit_mats[unit_column] = target_unit

    # Concatenate converted values with rest of original dataframe
    return pd.concat((mat_df[mat_df[unit_column] != unit_name_in_col], unit_mats)).sort_values(['Code','Type'])

In [ ]:
## Capacity unit conversions -> So all values are unit/tonne
""" BTL is for 2 types of plastic bottles - ignored for now
    M3 is only for water therefore 1000 KG equivalent and is for MM3 so 1/1000 for kg
    NM3 is for gases per MNM3 - conversions and to get tonne equivalent"""

# BTL filt
materials_filt = materials[materials['Capacity unit'] != 'MM BTL/yr']

# NM3 filt
unit_conversion = pd.read_excel(unit_conversion_path)
materials_filt = convert_units(materials_filt, unit_conversion, 'NM3', 'Capacity unit', 'Target/Process', 'MM NM3/yr', 'MM KG/yr', inv_density=True, standard_conversion=1)

# M3 filt
materials_filt['Value'] = materials_filt['Capacity unit'].replace({'MM KG/yr':1, 'MM M3/yr':1/1000}).astype(float)*materials_filt['Value'].astype(float)
materials_filt['Capacity unit'] = 'MM KG/yr'

In [ ]:
## Value unit conversions -> So all values are in kg/kg

# Filter out EA and BOAT
materials_values = materials_filt[materials_filt['Value unit'] != 'EA']
materials_values = materials_values[materials_values['Value unit'] != 'EA/TONNE']
materials_values = materials_values[materials_values['Value unit'] != 'BOAT']

# NM3 conversion
materials_values = convert_units(materials_values, unit_conversion, 'NM3', 'Value unit', 'Source/Object', 'NM3', 'kg/kg', standard_conversion=1000)
materials_values = convert_units(materials_values, unit_conversion, 'NM3', 'Value unit', 'Source/Object', 'NM3/TONNE', 'kg/kg', standard_conversion=1000)

# MMCAL conversion
materials_values = convert_units(materials_values, unit_conversion, 'MMCAL', 'Value unit', 'Source/Object', 'MMCAL', 'kg/kg', standard_conversion=1000)
materials_values = convert_units(materials_values, unit_conversion, 'MMCAL', 'Value unit', 'Source/Object', 'MMCAL/TONNE', 'kg/kg', standard_conversion=1000)

# Remaining standard conversions
conversion_ratios = {'TONNE/TONNE':1, 'TONNE':1, 'KG/TONNE':1E-3, 'KG':1E-3, 'G/TONNE':1E-6, 'G':1E-6, 'M3/TONNE':1, 'M3':1, 'KWH/TONNE':1E-3, 'KWH':1E-3, 'kg/kg':1}
conversion_names = {'TONNE/TONNE':'kg/kg', 'TONNE':'kg/kg', 'KG/TONNE':'kg/kg', 'KG':'kg/kg', 'G/TONNE':'kg/kg', 'G':'kg/kg', 'M3/TONNE':'kg/kg', 'M3':'kg/kg', 'KWH/TONNE':'kWh/kg',  'KWH':'kWh/kg', 'kg/kg':'kg/kg'}
materials_values['Value'] = materials_values['Value unit'].replace(conversion_ratios)*materials_values['Value']
materials_values['Value unit'] = materials_values['Value unit'].replace(conversion_names)

In [ ]:
# Add uncertainty
uncertainty_factor = 0.05
materials_values['Value_sigma'] = abs(materials_values['Value'])*uncertainty_factor

In [ ]:
# Output process recipes with consistent units and uncertainties
materials_values.reset_index(drop=True).to_csv(output_path+'ihsMaterials_w_uncertainties.csv')